# Motif Detection of Every Gene Detected in Experimental Data

Teague McCracken 
03/30/2025

In [50]:
# Initialize the notebook, set environment to transcriptomics
import pandas as pd
import subprocess
import os
import multiprocessing
from pathlib import Path
import numpy as np

data_path = '/home/temccrac/Programs/git_clones/ECE759_Project/cellular_clarity'
checkpoint_path = '/home/temccrac/Programs/cellular_clarity_project/checkpoints' # on Teague's server 

## Create the New Reference Genome, trying Araport11

## Step 1 - Extract promoter sequences for each gene

Inputs needed: <br>
1. gtf_file (reference genome information) <br>
2. gene_list_file (detected genes from experiment) <br>
3. genome.fa <br>
4. genomes size file Arabidopsis.genome (contains genome size information) <br>

Outputs: all outputs go to project checkpoint folder <br>
1. filtered_gtf (reduced version of Arabidopsis gtf file) <br>
2. tss.bed (bed format with transcription start sites) <br>
3. promoters.bed (bed format with 1000 bp upstream regions (promoters) of genes) <br>
4. promoters.fasta (the final output, sequences of promoter regions of each gene of interest) <br>

In [ ]:
# File paths for step 1, had 5324 missing genes
gtf = "/home/temccrac/Programs/data/genomes/Arabidopsis_thaliana.TAIR10.54.gtf"
#gene_list = "/home/temccrac/Programs/git_clones/ECE759_Project/cellular_clarity/genes_of_interest.txt" # our results
gene_list = "/home/temccrac/Programs/git_clones/ECE759_Project/cellular_clarity/paperresults/genes_of_interest.txt"
genome_fa = "/home/temccrac/Programs/data/genomes/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa"
genome_sizes = "/home/temccrac/Programs/data/genomes/Arabidopsis.genome"

In [ ]:
# File paths for step 1 using Araport 11 annotations, 2016, had ~2778 missing genes, only 35 are DEGs
gtf = "/home/temccrac/Programs/data/genomes/Araport11/Araport11_renamed.gff"
#gene_list = "/home/temccrac/Programs/git_clones/ECE759_Project/cellular_clarity/genes_of_interest.txt" # our results
gene_list = "/home/temccrac/Programs/git_clones/ECE759_Project/cellular_clarity/paperresults/genes_of_interest.txt"
genome_fa = "/home/temccrac/Programs/data/genomes/Araport11/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa"
genome_sizes = "/home/temccrac/Programs/data/genomes/Araport11/TAIR10.chrom.sizes"

In [ ]:
# File paths for step 1 using Araport 11 Mar9 2021 annotations, missing 2900 genes
gtf = "/home/temccrac/Programs/data/genomes/Araport11_2021/Araport11_renamed.gff"
#gene_list = "/home/temccrac/Programs/git_clones/ECE759_Project/cellular_clarity/genes_of_interest.txt" # our results
gene_list = "/home/temccrac/Programs/git_clones/ECE759_Project/cellular_clarity/paperresults/genes_of_interest.txt"
genome_fa = "/home/temccrac/Programs/data/genomes/Araport11_2021/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa"
genome_sizes = "/home/temccrac/Programs/data/genomes/Araport11_2021/TAIR10.chrom.sizes"

In [40]:
# File paths for step 1 using Araport 11 Feb 2022 annotations
gtf = "/home/temccrac/Programs/data/genomes/Araport11_2021/Araport11_renamed.gff"
#gene_list = "/home/temccrac/Programs/git_clones/ECE759_Project/cellular_clarity/genes_of_interest.txt" # our results
gene_list = "/home/temccrac/Programs/git_clones/ECE759_Project/cellular_clarity/paperresults/genes_of_interest.txt"
genome_fa = "/home/temccrac/Programs/data/genomes/Araport11_2021/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa"
genome_sizes = "/home/temccrac/Programs/data/genomes/Araport11_2021/TAIR10.chrom.sizes"

In [41]:
# Core functions for promoter extraction
def filter_gtf_by_genes(gtf_file, gene_list_file, output_gtf):
    with open(gene_list_file, 'r') as f:
        genes = set(f.read().splitlines())

    with open(gtf_file, 'r') as gtf_in, open(output_gtf, 'w') as gtf_out:
        for line in gtf_in:
            if line.startswith('#'):
                continue
            if any(gene in line for gene in genes):
                gtf_out.write(line)

def extract_tss_bed_original(filtered_gtf, output_bed): #with tair .gtf annotations
    awk_cmd = (
        '''awk '$3 == "transcript" {
            match($0, /gene_id "([^"]+)"/, m);
            gene = m[1];
            if ($7 == "+") {
                print $1 "\\t" $4-1 "\\t" $4 "\\t" gene "\\t.\\t" $7;
            } else {
                print $1 "\\t" $5-1 "\\t" $5 "\\t" gene "\\t.\\t" $7;
            }
        }' '''
    )
    full_cmd = f"{awk_cmd} {filtered_gtf} > {output_bed}"
    subprocess.run(full_cmd, shell=True, check=True)

def extract_tss_bed(filtered_gtf, output_bed): # command for using araport11 .gff
    awk_cmd = (
    '''awk '$3 == "mRNA" {
        # Extract the gene ID from "Parent=..." in the 9th field
        match($9, /Parent=([^;]+)/, m);
        gene = m[1];

        # For plus-strand, TSS is the start ($4). For minus-strand, TSS is the end ($5).
        if ($7 == "+") {
            print $1 "\\t" $4-1 "\\t" $4 "\\t" gene "\\t.\\t" $7;
        } else {
            print $1 "\\t" $5-1 "\\t" $5 "\\t" gene "\\t.\\t" $7;
        }
    }' '''
    )
    full_cmd = f"{awk_cmd} {filtered_gtf} > {output_bed}"
    subprocess.run(full_cmd, shell=True, check=True)

def create_promoter_bed(tss_bed, genome_sizes, output_bed, length=1000):
    with open(output_bed, 'w') as out_f:
        subprocess.run([
            "bedtools", "flank",
            "-i", tss_bed,
            "-g", genome_sizes,
            "-l", str(length),
            "-r", "0",
            "-s"
        ], stdout=out_f, check=True)

def extract_promoter_fasta(genome_fa, promoter_bed, output_fasta):
    subprocess.run([
        "bedtools", "getfasta",
        "-fi", genome_fa,
        "-bed", promoter_bed,
        "-fo", output_fasta,
        "-s",
        "-name"  # preserves gene IDs in FASTA headers
    ], check=True)

# Functions to execute the above commands in parallel
def run_promoter_pipeline(chunk_id, gene_list_chunk, gtf, genome_fa, genome_sizes, outdir):
    filtered_gtf = f"{outdir}/filtered_{chunk_id}.gtf"
    tss_bed = f"{outdir}/tss_{chunk_id}.bed"
    promoter_bed = f"{outdir}/promoters_{chunk_id}.bed"
    fasta_out = f"{outdir}/promoters_{chunk_id}.fa"

    # Write chunked gene list
    gene_list_file = f"{outdir}/genes_{chunk_id}.txt"
    with open(gene_list_file, 'w') as f:
        f.write('\n'.join(gene_list_chunk))

    # Run sub-steps
    filter_gtf_by_genes(gtf, gene_list_file, filtered_gtf)
    extract_tss_bed(filtered_gtf, tss_bed)
    create_promoter_bed(tss_bed, genome_sizes, promoter_bed)
    extract_promoter_fasta(genome_fa, promoter_bed, fasta_out)

def parallel_promoter_extraction(genes_file, gtf, genome_fa, genome_sizes, outdir, num_cpus=10):
    with open(genes_file) as f:
        genes = f.read().splitlines()

    # Split gene list into chunks
    chunk_size = len(genes) // num_cpus
    chunks = [genes[i:i + chunk_size] for i in range(0, len(genes), chunk_size)]

    os.makedirs(outdir, exist_ok=True)

    args = [(i, chunks[i], gtf, genome_fa, genome_sizes, outdir) for i in range(len(chunks))]
    with multiprocessing.Pool(num_cpus) as pool:
        pool.starmap(run_promoter_pipeline, args)

    # Merge all .fa files
    merged_fasta = os.path.join(outdir, "promoters_merged.fa")
    with open(merged_fasta, 'w') as outfile:
        for i in range(len(chunks)):
            chunk_fasta = os.path.join(outdir, f"promoters_{i}.fa")
            with open(chunk_fasta, 'r') as infile:
                outfile.write(infile.read())

    print("✅ All promoters extracted and merged.")

In [45]:
# Run steps for Step 1
parallel_promoter_extraction(
    genes_file=gene_list,
    gtf=gtf,
    genome_fa=genome_fa,
    genome_sizes=genome_sizes,
    outdir=checkpoint_path,
    num_cpus=10
)

print("✅ Promoter FASTA extraction complete.")

✅ All promoters extracted and merged.
✅ Promoter FASTA extraction complete.


After running, I deleted all the unneccessary intermediate files and kept only the promoter_merged.fa file

Now, I check that all the genes of interests have promoters that were saved in the merged promoter file

In [47]:
def check_promoter_coverage(gene_list_file, promoters_fasta):
    # Read expected gene list
    with open(gene_list_file) as f:
        expected_genes = set(f.read().splitlines())

    # Read FASTA headers and extract gene IDs from lines like "AT5G23180::5:..."
    with open(promoters_fasta) as f:
        found_genes = set(
            line[1:].strip().split("::")[0]
            for line in f if line.startswith(">")
        )

    missing = expected_genes - found_genes
    extra = found_genes - expected_genes

    print(f"✅ Found promoters for {len(found_genes)} / {len(expected_genes)} genes.")
    if missing:
        print(f"❌ Missing {len(missing)} genes:")
        for gene in list(missing)[:10]:
            print("  ", gene)
    else:
        print("🎉 No missing genes.")

    if extra:
        print(f"⚠️ {len(extra)} unexpected gene(s) in output (not in your original list):")
        for gene in list(extra)[:5]:
            print("  ", gene)
    return missing


# Example usage:
missing = check_promoter_coverage(
    gene_list_file=gene_list,
    promoters_fasta="/home/temccrac/Programs/cellular_clarity_project/checkpoints/promoters_merged.fa"
)

✅ Found promoters for 30824 / 33602 genes.
❌ Missing 2778 genes:
   AT2G15950
   AT4G04693
   AT3G30147
   AT2G25400
   AT3G44030
   AT2G07742
   AT3G22492
   AT1G28870
   AT5G34903
   AT4G13611


In [49]:
len(missing)

2778

In [51]:
DEGs_pr = pd.read_csv(os.path.join(data_path, 'paperresults/DEGs_clusters_epidermis_paperresults.csv'), index_col=None) # We are working with the Epidermis data

In [53]:
notmissing = set(DEGs_pr['AGI']) - set(missing) 
len(notmissing)

2704

In [ ]:
missing_DEGs = set(missing).intersection(set(DEGs_pr['AGI']))
missing_DEGs

35

## Step 2 - Clustering / DEG Detection

I will use the papers DEG and cluster results

## Step 3 - Schmittling Motif Detection Approach <br>
### Scan promoter sequences from each DPGP cluster for enriched motifs by comparing to a random subset of non-DEG genes promoters (of equal size)

In [ ]:
def max_normalize_expression(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normalize gene expression values using the formula from Schmittling et al. (2023).
    
    Parameters:
        df (pd.DataFrame): Rows are genes, columns are time points. Values are RPKM.
    
    Returns:
        pd.DataFrame: Normalized gene expression matrix.
    """
    # Mean expression per gene (across all time points)
    gene_mean = df.mean(axis=1)

    # Normalize as per equation: (R_t - mean(R)) / R_t
    normalized_df = df.sub(gene_mean, axis=0).div(df, axis=0)
    
    # Replace any remaining inf or NaN to handle 0 expression values
    normalized_df.replace([np.inf, -np.inf], 0, inplace=True)
    normalized_df.fillna(0, inplace=True)

    return normalized_df

In [ ]:
# Perform DPGP clustering of DEGs 
DEGs_path = os.path.join(data_path, 'unique_DEGs_of_interest.txt')
with open(DEGs_path) as f:
    DEGs_list = f.read().splitlines()
rpkm_values = pd.read_csv(os.path.join(data_path, 'rpkm_values.csv'), index_col = 0)
norm_rpkm = max_normalize_expression(rpkm_values)
norm_rpkm.loc[DEGs_list]

,E - 1,B - 2,D - 1,G - 2,C - 2,D - 2,A1,G + 4,D + 3,A2,...,G - 1,C - 1,C + 3,F + 1,E + 1,E + 3,E - 3,D - 3,B - 1,C + 1
AT1G80240,0.335920,0.185114,0.287347,0.255676,-0.078306,0.205573,0.201530,-0.227410,-0.267387,0.049417,...,0.138980,0.072828,-0.176348,-0.156141,-0.206972,-0.702269,0.369182,0.214882,0.171156,-0.842016
AT4G22690,-6.710523,-0.315769,-3.224730,-4.085675,-9.807533,-46.819983,-29.918344,-4.389371,-1.956036,-6.937880,...,-7.565493,-0.945380,0.234551,-4.044779,-1.563933,-0.740192,0.000000,0.000000,0.397273,0.642348
AT2G41380,0.287610,0.379133,0.489668,0.494602,0.245074,0.126963,0.417892,-1.650569,-0.730997,0.464912,...,0.380743,0.318087,-3.118281,-3.138676,-0.865297,-7.387341,0.423541,0.313138,0.250814,-5.033319
AT1G67980,-1.831027,0.144852,-2.554763,-4.477346,-4.819935,-6.573944,-4.203048,-1.790592,-6.959236,-3.274607,...,-3.324295,-3.029236,0.377812,-1.936920,-1.832199,-0.354852,-11.438597,-4.893797,0.302454,0.583034
AT5G58784,0.222754,-0.076038,0.152269,-0.037666,-0.018905,0.160477,0.100276,0.060175,0.105228,-0.039351,...,-0.117344,0.049161,-0.103114,0.120344,-0.089168,-0.252786,0.274902,0.220990,0.004727,-0.701032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AT1G02080,0.084232,-0.280294,0.131909,-0.413874,0.154386,0.219653,-0.407040,-0.308211,0.176434,-0.303817,...,-0.320018,0.246330,-0.346617,-0.035909,-0.004250,0.136810,-0.150180,0.000460,-0.092738,-0.264360
AT5G40560,-5.200254,0.382806,-0.887349,-0.363179,0.818945,0.465925,0.171256,0.000000,0.427751,-1.127693,...,-2.443880,0.627539,0.000000,-0.081774,0.083674,-0.865785,0.009387,-0.833530,-2.958136,0.000000
AT1G02890,0.254786,0.002798,-0.070132,0.124736,0.075224,0.007382,-0.166204,-0.228460,0.047540,0.179140,...,0.111822,0.075829,-0.143779,-0.253290,-0.168911,-0.311604,-0.200288,0.053404,-0.102670,-0.587922
AT1G18990,0.068405,-0.303578,0.108185,0.191676,0.220335,-0.032927,0.073633,-0.564288,-0.165482,0.169261,...,-0.021058,0.269941,-0.083367,-0.298191,-0.501583,-0.198194,0.029642,-0.084329,0.070128,-0.550231


In [53]:
from DP_GP_cluster import dpgp

ModuleNotFoundError: No module named 'DP_GP_cluster'